In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/sample_submission.csv
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train.csv
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test.csv
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/08450.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/05938.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/06122.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/08536.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/04168.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/02371.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/04377.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/00767.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/07457.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/02360.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/05222.jpg
/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test/03999.jpg
/kaggle/input/

In [2]:
!pip install trackio
!pip install torchsummary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.0/875.0 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.5
    Uninstalling pydantic_core-2.41.5:
      Successfully uninstalled pydantic_core-2.41.5
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.4
    Uninstalling pydant

# Library Imports

In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import sklearn
import pandas as pd
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import torch
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from sklearn.metrics import f1_score, mean_squared_error
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau
import trackio
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize DataSet for Dataloader

In [4]:
class FaceDataSet(Dataset):
  def __init__(self, csv_path, dataset_path, transform):
    super().__init__()
    self.data = pd.read_csv(csv_path)
    self.dataset_path = dataset_path
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    #This is a dunder function because DataLoader calls it implicitly to get the data
    data_item = self.data.iloc[idx]
    image = Image.open(os.path.join(self.dataset_path, data_item['full_path'])).convert('RGB')
    if self.transform:
      image = self.transform(image)
    age = torch.tensor(data_item['age'], dtype=torch.float32)
    gender = torch.tensor(data_item['gender'], dtype=torch.long)
    return image, (age,gender)

# Load Dataset

In [5]:
# transforms on training data
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),   # image augmentation (flip)
    transforms.RandomRotation(degrees=10),    # image augmentation (rotation)
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])   #Using stats of ImageNet model
])

# transforms for testing/validation (no augmentation to be done)
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#dataset paths are initialized
train_csv_path = '/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train.csv'
dataset_path = '/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset'

test_csv_path = '/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test.csv'

train_data = FaceDataSet(train_csv_path, dataset_path, train_transforms)

#80% for training, 20% for validation
train_len = int(0.8 * len(train_data))
val_len = len(train_data) - train_len

# splitting training data for testing and validation 
train_set, val_set = random_split(train_data, [train_len, val_len])

# Dataloaders to load the train_set
train_loader = DataLoader(dataset=train_set,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2,  #number of parallel processes
                          pin_memory=True if device == 'cuda' else False)

val_loader = DataLoader(dataset=val_set,
                        batch_size=32,
                        num_workers=2,
                        pin_memory=True if device == 'cuda' else False)

# Define Model Architecture

In [ ]:
class FaceCNNModel (nn.Module):
  def __init__ (self):
    super(FaceCNNModel, self).__init__()

    # A neural network with 4 convolutional blocks
    self.conv1 = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv4 = nn.Sequential(
        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    # A fully connected layer that outputs 512 features
    self.flatten_dim = 256 * 14 * 14
    self.fc1 = nn.Sequential(
        nn.Linear(self.flatten_dim, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.4)
    )

    self.age_head = nn.Linear(512, 1)  # Regression Head which outputs just the age
    self.gender_head = nn.Linear(512, 2) # Gender Head which outputs two logits for male and female respectively

  def forward (self, x):
    # Defining a single forward pass
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = x.view(x.size(0), -1)
    x = self.fc1(x)
    age = self.age_head(x).squeeze(1)
    gender = self.gender_head(x)

    return age, gender


# Print Model Summary

In [7]:
model = FaceCNNModel().to(device)  # Printing Model Summary
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
       BatchNorm2d-2         [-1, 32, 224, 224]              64
              ReLU-3         [-1, 32, 224, 224]               0
         MaxPool2d-4         [-1, 32, 112, 112]               0
            Conv2d-5         [-1, 64, 112, 112]          18,496
       BatchNorm2d-6         [-1, 64, 112, 112]             128
              ReLU-7         [-1, 64, 112, 112]               0
         MaxPool2d-8           [-1, 64, 56, 56]               0
            Conv2d-9          [-1, 128, 56, 56]          73,856
      BatchNorm2d-10          [-1, 128, 56, 56]             256
             ReLU-11          [-1, 128, 56, 56]               0
        MaxPool2d-12          [-1, 128, 28, 28]               0
           Conv2d-13          [-1, 256, 28, 28]         295,168
      BatchNorm2d-14          [-1, 256,

# Training function

In [8]:
def train_epoch (train_batch, FaceModel, optimizer, criterion_age, criterion_gender):
  FaceModel.train()  #Sets model in train Mode
  images , (ages, genders) = train_batch
  images, ages, genders = images.to(device), ages.to(device), genders.to(device)
  optimizer.zero_grad()  #Initializes the gradients to zero
  age_out, gender_out = FaceModel(images)
  age_loss = criterion_age(age_out, ages)   #loss calc for age (Mean squared error)
  gender_loss = criterion_gender(gender_out, genders) #loss calc for gender (Cross Entropy loss)
  tot_loss = age_loss + gender_loss
  tot_loss.backward()   #Back propagation
  optimizer.step()    #Weights are adjusted

  return tot_loss.item()

# Validation function

In [9]:
def validate_epoch (validation_batch, FaceModel, criterion_age, criterion_gender):
  FaceModel.eval()  # Runs Model in evaluation mode
  images, (ages, genders) = validation_batch
  images, ages, genders = images.to(device), ages.to(device), genders.to(device)
  with torch.no_grad():   #Gradients disabled
    age_out, gender_out = FaceModel(images)
    age_loss = criterion_age(age_out, ages)
    gender_loss = criterion_gender(gender_out, genders)
    tot_loss = age_loss + gender_loss
    gender_pred = torch.softmax(gender_out, dim=1)
    gender_pred = torch.argmax(gender_pred, dim=1).cpu().numpy() # Gender prediction with softmax followed by argmax
    age_pred = torch.clamp(age_out, min=0, max=100).cpu().numpy() #age normalized
  return tot_loss.item(), gender_pred, age_pred


# Trackio Initialization

In [10]:
'''
run = trackio.init(
    project="25-t3-nppe1",
    space_id="bhavin273/dlgenai-nppe",
    name="FaceDataSet_CNN",
)
'''

'\nrun = trackio.init(\n    project="25-t3-nppe1",\n    space_id="bhavin273/dlgenai-nppe",\n    name="FaceDataSet_CNN",\n)\n'

# Model Training

In [11]:
def train (FaceCNNModel):
    num_epochs = 15
    patience_counter = 0
    patience_limit = 5
    best_score = float('-inf')
    model = FaceCNNModel.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    # scheduler to adjust the optimizer in case the curve flattens
    scheduler = ReduceLROnPlateau(optimizer=optimizer, mode='max', factor=0.5, patience=2, min_lr=1e-6)
    age_criterion = nn.MSELoss()
    gender_criterion = nn.CrossEntropyLoss()

    for epoch in range (num_epochs):
      tot_train_loss = 0
      # Training loop
      for images, (ages, genders) in train_loader:
        train_loss = train_epoch((images, (ages, genders)), model, optimizer, age_criterion, gender_criterion)
        tot_train_loss += train_loss * images.size(0)   # Batch mean loss multiplied by batch size for total batch loss
      epoch_train_loss = tot_train_loss/len(train_loader.dataset)  # Total Train loss per epoch
      tot_val_loss = 0
      age_pred_tot, gender_pred_tot = [], []
      age_true_tot, gender_true_tot = [], []
      # Validation loop
      for images, (ages, genders) in val_loader:
        age_true_tot.extend(ages.cpu().numpy())
        gender_true_tot.extend(genders.cpu().numpy())
        val_loss, gender_pred, age_pred = validate_epoch((images, (ages, genders)), model, age_criterion, gender_criterion)
        age_pred_tot.extend(age_pred)
        gender_pred_tot.extend(gender_pred)
        tot_val_loss += val_loss * images.size(0)
      epoch_validate_loss = tot_val_loss/len(val_loader.dataset)
      gender_score = f1_score(y_true=gender_true_tot, y_pred=gender_pred_tot, average='macro') # f1 macro for evaluating the gender score
      nrmse = np.sqrt(mean_squared_error(y_true=age_true_tot, y_pred=age_pred_tot))/100 # using nrmse for age
      age_score = 1 - min(nrmse, 30)/30
      tot_score = 2 * (age_score*gender_score) / (age_score+gender_score)  #Harmonic mean of age and gender score
      current_lr = optimizer.param_groups[0]['lr']
  

      print(f'Epoch {epoch+1}/{num_epochs}')
      print(f'Train Loss : {epoch_train_loss:.4f} | Validation Loss : {epoch_validate_loss:.4f}')
      print(f'Age Score : {age_score:.4f} | Gender Score : {gender_score:.4f} | total score : {tot_score:.4f}')
      print(f'Current learning Rate is : {current_lr:.6f}\n')

      # Logging metrics to trackio
      run.log({
        "epoch": epoch + 1,
        "train_loss": epoch_train_loss,
        "val_loss": epoch_validate_loss,
        "age_score": age_score,
        "f1_gender": gender_score,
        "total_score": tot_score,
        "learning_rate": current_lr
      })

        
      scheduler.step(tot_score)
      if tot_score > best_score:
        patience_counter = 0
        best_score = tot_score
        torch.save(model.state_dict(), 'best_finetuned_model.pth')   # save model
      else:
        patience_counter += 1
  
      if patience_counter >= patience_limit:
        print('-----Early Stopping------')
        break



# Prediction

In [12]:
test_df = pd.read_csv(test_csv_path)

class TestDataSet (Dataset):
    def __init__ (self, test_csv_path, dataset_path, transform):
        self.data = pd.read_csv(test_csv_path)
        self.dataset_path = dataset_path
        self.transform = transform

    def __len__(self):
        return len (self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]
        image = Image.open(os.path.join(self.dataset_path, image_path['full_path'])).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

test_data = TestDataSet(test_csv_path, dataset_path, test_transforms)

test_loader = DataLoader(dataset=test_data,
                        batch_size=32,
                        num_workers=2,
                        pin_memory=True if device == 'cuda' else False)

In [13]:
age_pred, gender_pred = [], []
def predict (model_name, model_path):
    model = model_name.to(device)
    model.load_state_dict(torch.load(model_path))

    model.eval()

    for images in test_loader:
        images = images.to(device)
        with torch.no_grad():
            ages, genders = model (images)
            age_out = torch.clamp(ages, min=0, max=100).cpu().numpy()
            gender_out = torch.softmax(genders, dim=1)
            gender_out = torch.argmax(gender_out, dim=1).cpu().numpy()
            age_pred.extend(age_out.tolist())
            gender_pred.extend(gender_out.tolist())


# Finetuning ResNet Model

In [14]:
class FaceResNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(weights='IMAGENET1K_V1')
        self.backbone = nn.Sequential(*list(base.children())[:-1])  # frozen the convolutional base
        self.fc1 = nn.Sequential(
            nn.Linear(512, 256), # 256 output channels this time
            nn.ReLU(),
            nn.Dropout(0.3) # Reduced the dropout rate compared to the previous model
        )
        self.age_head = nn.Linear(256, 1)
        self.gender_head = nn.Linear(256, 2)

    def forward(self, x):
        x = self.backbone(x)             
        x = x.view(x.size(0), -1)        
        x = self.fc1(x)
        age = self.age_head(x).squeeze(1)
        gender = self.gender_head(x)
        return age, gender

In [15]:
#train(FaceResNetModel())


In [16]:
predict(FaceResNetModel(),'/kaggle/input/finetuned-resnet/pytorch/default/1/best_finetuned_model.pth')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 199MB/s]


# Generating Submission File

In [17]:

submission_path = '/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/sample_submission.csv'
submission_df = pd.read_csv(submission_path)
test_ids = submission_df['id']
submission = pd.DataFrame({
    "id": test_ids,
    "age": age_pred,
    "gender": gender_pred
})

submission.to_csv("/kaggle/working/submission.csv", index=False)



In [18]:
submission.head()

,id,age,gender
0,0,29.050686,1
1,1,25.693245,1
2,2,26.110558,1
3,3,31.523764,1
4,4,72.011444,1
